In [13]:
import numpy as np
import math
import scipy

In [21]:
Ns = 8
J = 1.0
g = 0.4

In [22]:
#flip nth bit
def FlipBit(i,n):
    return i^(1<<n)

#read nth bit
def ReadBit(i,n):
    return (i&(1<<n))>>n

#pick up n bits from kth bit
def PickBit(i,k,n):
    return (i&((2**n-1)<<k))>>k

#circuilar bit shift right
def RotLBit(i,L):
    return (PickBit(i,0,L-1)<<1)+(i>>(L-1))

#define a function to get all the configurations that can be obtained by translation of i
def transl_bit_patterns(i):
    transl_bits = [i]
    for _ in range(Ns-1):
        i = RotLBit(i,Ns)
        transl_bits.append(i)
    return transl_bits

In [23]:
#create the check list for whether it is a representative configuration and its corresponding representative config \r>
Check = np.zeros([2**Ns,3])
all_configs = []
for k in range(2**Ns):
    if k in all_configs:
        Check[k,1] = min(transl_bit_patterns(k))
        Check[k,2] = transl_bit_patterns(min(transl_bit_patterns(k))).index(k)
    else:
        all_configs += transl_bit_patterns(k)
        Check[k,0] = 1
        Check[k,1] = k

In [24]:
#create all the representative configurations using the checklist
rep_configs = []
for i in range(2**Ns):
    if Check[i,0] == 1:
        rep_configs.append(i)

#create the basis for every k, allow mod(kR,Ns) = 0, where R is the minimal translating times needed to obtain the original state
all_basis = []
for k in range(Ns):
    k_basis = []
    for r in rep_configs:
        if k*(Ns/transl_bit_patterns(r).count(r)) % Ns == 0:
            k_basis.append((k,r))
    all_basis.append(k_basis)

In [25]:
#applying Hamiltonian to the representative configurations |r>
def apply_Hamil(r):
    Ham_r = []
    couplings = 0
    for i in range(Ns):
        Ham_r.append((-J*g,FlipBit(r,i)))
        couplings += (-1)**(ReadBit(r,i%Ns)+ReadBit(r,(i+1)%(Ns)))
    Ham_r.append((-J*couplings,r))
    return Ham_r

#applying Pk to the representative configurations |r>
def apply_proj(k,r):
    Proj_r = []
    for j in range(Ns):
        Proj_r.append((np.exp(1j*2*math.pi*k*j/Ns),transl_bit_patterns(r)[j]))
    return Proj_r

#calculating <s|P_k H|r>
def mel_Pk_H(k,r,s):
    mel = 0
    for entries in apply_Hamil(r):
        for m in range(len(apply_proj(k,s))):
            if apply_proj(k,s)[m][1] == entries[1]:
                mel += apply_proj(k,s)[m][0]*entries[0]
    return(mel)

#calculating <r|P_k|r>
def ev_Pk_r(k,r):
    ev = 0
    for entries in apply_proj(k,r):
        if entries[1] == r:
            ev += entries[0]
    return(ev)

In [26]:
#generate each k-sector using the function defined above and diagonalize each block
for i in range(Ns):
    H_matrix = np.zeros([len(all_basis[i]),len(all_basis[i])])+0j
    for row_idx in range(len(all_basis[i])):
        for col_idx in range(len(all_basis[i])):
            r = all_basis[i][col_idx][1]
            s = all_basis[i][row_idx][1]
            mel = mel_Pk_H(i,r,s)/np.sqrt(np.real((ev_Pk_r(i,s))*(ev_Pk_r(i,r))))
            H_matrix[row_idx,col_idx] = mel
    evals,evecs = np.linalg.eigh(H_matrix)
    print("(1)_Momentum_Sector_ki={}:{}".format(i,np.sort(evals)))      

(1)_Momentum_Sector_ki=0:[-8.32346445e+00 -8.32320887e+00 -5.72840338e+00 -5.23953684e+00
 -4.62729724e+00 -4.01507702e+00 -3.48008258e+00 -3.06859501e+00
 -2.81114570e+00 -2.03223617e+00 -9.31404990e-01 -8.85021510e-01
 -8.00000000e-01 -8.00000000e-01 -2.16084628e-01 -1.54065923e-01
 -1.54065923e-01 -5.74868673e-16 -2.90876444e-16 -6.80028190e-17
  2.53736334e-16  1.50770223e-02  2.16084628e-01  8.00000000e-01
  8.00000000e-01  8.85021510e-01  1.23953684e+00  2.03223617e+00
  2.81114570e+00  3.48008258e+00  4.15406592e+00  4.15406592e+00
  4.32320887e+00  4.62729724e+00  5.72840338e+00  8.32346445e+00]
(1)_Momentum_Sector_ki=1:[-5.58137285e+00 -5.17785031e+00 -4.62730693e+00 -4.05368991e+00
 -3.54183602e+00 -3.14561414e+00 -2.89590194e+00 -1.45862884e+00
 -1.27324101e+00 -6.27306929e-01 -5.50553069e-01 -4.58163984e-01
 -3.34468441e-01 -3.26758994e-01 -5.93872565e-16  5.69445835e-16
  1.87770093e-01  3.34468441e-01  4.58163984e-01  5.50553069e-01
  6.27306929e-01  1.41222991e+00  1.458